<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/Japanese_Stable_LM_2_Base_1_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Japanse Stable LM 2 1.6B on Colab by AICU

これは Stability AI が2024/5/9にリリースした日本語大規模言語モデル「[Japanese Stable LM 2 1.6B](https://ja.stability.ai/blog/japanese-stable-lm-2-16b)」に関するGoogle Colabでのサンプルコードです。

T4 GPUなどGPUあり環境で実行してみてください（CPUでも動きます）。

解説記事はこちら

**Stability AI、日本語大規模言語モデル「Japanese Stable LM 2 1.6B」を発表。Google Colabのシークレット機能で使ってみた**
https://note.com/aicu/n/nc4ebdc2a3eeb

In [ ]:
#!pip install transformers==4.40.1
!pip install huggingface_hub
!pip install torch
!pip install tiktoken
!pip install accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 左側の「シークレット」から「HF_TOKEN」という環境変数を設定指定ください
# https://twitter.com/GoogleColab/status/1719798406195867814
# コードに直書きしたいひとは以下のように書いても動くと思います
# token="(あなたのHugginFaceトークン)"
# !huggingface-cli login --token $token

token = userdata.get('HF_TOKEN')


In [ ]:
model_name = "stabilityai/japanese-stablelm-2-base-1_6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    trust_remote_code=True,
#   torch_dtype=torch.float16,
    torch_dtype=torch.float32,  # ここをfloat32に変更しました
    low_cpu_mem_usage=True,
    device_map="auto"
)

prompt = """
AI で科学研究を加速するには、
""".strip()

inputs = tokenizer(
    prompt,
    add_special_tokens=True,
    return_tensors="pt"
).to(model.device)

# this is for reproducibility.
# feel free to change to get different result
seed = 23
torch.manual_seed(seed)

tokens = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


In [ ]:
# シードは固定すると同じ結果が出力されます。以下の2行をコメントアウトすると毎回異なる結果になります。
seed = 39
torch.manual_seed(seed)
# 自分のテキストで推論してみよう
prompt = """
吾輩は猫である、名前は…
""".strip()

inputs = tokenizer(
    prompt,
    add_special_tokens=True,
    return_tensors="pt"
).to(model.device)

tokens = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


時同じくして、アドボケーター仲間のDELLさんが Japanese Stable LM 2 Instruct 1.6B の Google Colab版コードを公開してくれています。

https://twitter.com/xqdior/status/1788459275809067424

Gradioインタフェースで日本語チャットを楽しめます！